In [1]:
from os import listdir, getcwd, rename, makedirs
from os.path import isfile, join, isdir, exists
import pandas as pd
import logging

log = logging.getLogger(__name__)
logging.basicConfig(filename='merge_all_data_to_one.log', level=logging.INFO, format='%(levelname)s:%(asctime)s: %(message)s', datefmt='%d/%m/%Y %I:%M %p')
start_dir = getcwd()
home = "/Users/tushar/PDX/pdxfinder-data/data/UPDOG"

In [2]:
def get_dirs(path):
    return [f for f in listdir(path) if isdir(join(path, f))]

def get_files(path):
    return [f for f in listdir(path) if isfile(join(path, f))]

def merge_data(path, data):
    tsv_files = [f for f in get_files(path) if f.endswith('.tsv')]
    if len(tsv_files)>0:
        for f in tsv_files:
            if f.__contains__('patient.tsv') or f.__contains__('patient_sample.tsv') or f.__contains__('pdx_model.tsv') or f.__contains__('model_validation.tsv') or f.__contains__('cell_model.tsv') or f.__contains__('sharing.tsv'):
                #log.info("\t\tReading File: "+ f)
                metadata = pd.read_csv(join(path,f), sep='\t', na_values="", low_memory=False)
                if 'Field' in metadata.columns:
                    metadata = metadata.loc[metadata.Field.astype('str').str.startswith('#') != True,].reset_index(drop=True)
                    metadata = metadata.drop('Field', axis=1)
                #metadata = metadata.iloc[4:,]
                if f.__contains__('patient.tsv'):
                    data[0] = data[0]._append(metadata)
                    log.info("\tFile: "+ f)
                elif f.__contains__('patient_sample.tsv'):
                    data[1] = data[1]._append(metadata)
                    log.info("\tFile: "+ f)
                elif f.__contains__('pdx_model.tsv'):
                    data[2] = data[2]._append(metadata)
                    log.info("\tFile: "+ f)
                elif f.__contains__('model_validation.tsv'):
                    data[3] = data[3]._append(metadata)
                    log.info("\tFile: "+ f)
                elif f.__contains__('cell_model.tsv'):
                    data[4] = data[4]._append(metadata)
                    log.info("\tFile: "+ f)
                elif f.__contains__('sharing.tsv'):
                    data[5] = data[5]._append(metadata)
                    log.info("\tFile: "+ f)
                metadata = pd.DataFrame()

    else:
        log.info("No .xlsx file found.")
    return  data

patient, ps, pdx, mv, cm, share = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
data = [patient, ps, pdx, mv, cm, share]
for provider in get_dirs(home): ## get_dirs will get the provider dirs in updog
    log.info("Working on provider: "+provider)
    data = merge_data(join(home, provider), data)

In [3]:
with pd.ExcelWriter('/Users/tushar/PDX/SA-Y3/validation_all_data.xlsx') as writer:
    data[0].to_excel(writer, sheet_name='patient', index=False)
    data[1].to_excel(writer, sheet_name='patient_sample', index=False)
    data[2].to_excel(writer, sheet_name='pdx_model', index=False)
    data[3].to_excel(writer, sheet_name='model_validation', index=False)
    data[4].to_excel(writer, sheet_name='cell_model', index=False)
    data[5].to_excel(writer, sheet_name='sharing', index=False)